[github link](https://github.com/ilyakava/sumproduct)

## Interface
```python
g = FactorGraph()

x1 = Variable('x1', 2) # init a variable with 2 states
x2 = Variable('x2', 3) # init a variable with 3 states

f12 = Factor('f12', np.array([
  [0.8,0.2],
  [0.2,0.8],
  [0.5,0.5]
])) # create a factor, node potential for p(x1 | x2)

# connect the parents to their children
g.add(f12)
g.append('f12', x2) # order must be the same as dimensions in factor potential!
g.append('f12', x1) # note: f12 potential's shape is (3,2), i.e. (x2,x1)

g.compute_marginals() # -> [0.0]

g.nodes['x1'].marginal() # -> array([0.5, 0.5])
```

In [4]:
import numpy as np

In [27]:
isinstance(np.array([]), np.ndarray)

True

In [42]:
class Variable:
    def __init__(self, name, n_states):
        self.name = name
        self.states = list(range(n_states))
    
    def __repr__(self):
        return self.name

In [46]:
class Factor:
    def __init__(self, name, array):
        assert isinstance(array, np.ndarray), f'´array has to be a numpy ndarray, not {type(array)}'
        self.name = name
        self.values = array
        self.shape  = self.values.shape
    
    def __repr__(self):
        return self.name

In [34]:
f = Factor('f12', [])
isinstance(f, Variable)

AssertionError: ´array has to be a numpy ndarray, not <class 'list'>

In [64]:
class FactorGraph:
    def __init__(self):
        self.nodes = {}
        self.factors = {}
        self.factor_vars = {}
        self.vars = {}
        self.var_factors = {}
    
    def add(self, factor):
        """Add a factor to the graph"""
        assert factor.name not in self.factors, f'Factor with name {factor.name} already defined'
        self.factors[factor.name] = factor
        self.factor_vars[factor.name] = []
    
    def append(self, factor, variable):
        """Add a variable to factor"""
        assert factor in self.factors, f'Factor {factor} not yet defined'
        expected_size = self.factors[factor].shape[len(self.factor_vars[factor])]
        print(f'Expected size = {expected_size}, true size = {len(variable.states)}')
        assert expected_size == len(variable.states)
        if variable.name not in self.vars:
            self.vars[variable.name] = variable
            self.var_factors[variable.name] = []
        self.factor_vars[factor].append(variable)
        self.var_factors[variable.name].append(self.factors[factor])
    
    def neighbours(self, entity):
        if isinstance(entity, Variable):
            return self._var_neighbours(entity)
        elif isinstance(entity, Factor):
            return self._factor_neighbours(entity)
        else:
            assert False, f'Illegal type {type(entity)}'
    
    def _var_neighbours(self, var):
        assert var.name in self.var_factors, f'unknown variable {var.name}'
        return self.var_factors[var.name]
    
    def _factor_neighbours(self, factor):
        assert factor.name in self.factor_vars, f'unknown factor {factor.name}'
        return self.factor_vars[factor.name]
        
    
    def compute_marginals(self):
        pass

In [65]:
## test
g = FactorGraph()

x1 = Variable('x1', 2) # init a variable with 2 states
x2 = Variable('x2', 3) # init a variable with 3 states

f12 = Factor('f12', np.array([
  [0.8,0.2],
  [0.2,0.8],
  [0.5,0.5]
])) # create a factor, node potential for p(x1 | x2)

# connect the parents to their children
g.add(f12)
g.append('f12', x2) # order must be the same as dimensions in factor potential!
g.append('f12', x1) # note: f12 potential's shape is (3,2), i.e. (x2,x1)

#g.compute_marginals() # -> [0.0]

#g.nodes['x1'].marginal() # -> array([0.5, 0.5])



Expected size = 3, true size = 3
Expected size = 2, true size = 2


In [66]:
g.neighbours(x1)

[f12]

In [67]:
## second test
g = FactorGraph()

A = Variable('A', 2)
B = Variable('B', 2)
C = Variable('C', 2)
D = Variable('D', 2)

f1 = Factor('f1', np.array([
    [10, 1],
    [1, 10]
]))

f2 = Factor('f2', np.array([
    [1, 10],
    [10, 1]
]))

f3 = Factor('f3', np.array([
    [10, 1],
    [1, 10]
]))

f4 = Factor('f4', np.array(
    [10, 1]
))

g.add(f1)
g.append('f1', A)
g.append('f1', B)

g.add(f2)
g.append('f2', B)
g.append('f2', C)

g.add(f3)
g.append('f3', B)
g.append('f3', D)

g.add(f4)
g.append('f4', C)

Expected size = 2, true size = 2
Expected size = 2, true size = 2
Expected size = 2, true size = 2
Expected size = 2, true size = 2
Expected size = 2, true size = 2
Expected size = 2, true size = 2
Expected size = 2, true size = 2


In [68]:
g.factors

{'f1': f1, 'f2': f2, 'f3': f3, 'f4': f4}

In [69]:
g.neighbours(f4)

[C]